# Импорт

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from LookGenerator.datasets.person_segmantation_dataset import PersonSegmentationDataset, PersonSegmentationDatasetMultichannel
from LookGenerator.networks.segmentation import UNet, train_unet

# Загрузка данных

In [3]:
transform_input = transforms.Compose([
    transforms.Resize((192, 256)),
    transforms.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=[0.5, 0.5, 0.5]
    )
])

transform_output = transforms.Compose([
    transforms.Resize((192, 256)),
    transforms.Normalize(
        mean=[0.5],
        std=[0.25]
    )
])

In [4]:
batch_size_train = 12
batch_size_val = 8

In [5]:
train_dataset = PersonSegmentationDatasetMultichannel("C:\\Users\\DenisovDmitrii\\Desktop\\train", transform_input=transform_input, transform_mask=transform_output)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
len(train_dataloader)

270

In [6]:
val_dataset = PersonSegmentationDatasetMultichannel("C:\\Users\\DenisovDmitrii\\Desktop\\test", transform_input=transform_input, transform_mask=transform_output)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size_val, shuffle=False)
len(val_dataloader)

103

Test load train_dataloader. and val.

In [7]:
for X, y in train_dataloader:
    print(X.shape)
    print(y.shape)
    break

torch.Size([12, 3, 256, 192])
torch.Size([12, 16, 256, 192])


# Обучение модели

Метки классов на изображении:

1. Левое лицо                       248,251,14
2. Правое лицо                      251,235,25
3. Туловище                         20,80,194
4. Внешняя верхняя левая рука       190,189,96
5. Внешняя верхняя правая рука      215,187,88
6. Внешняя нижняя левая рука        252,206,48
7. Внешняя нижняя правая рука       250,220,36
8. Внутренняя верхняя левая рука    145,191,116
9. Внутренняя верхняя правая рука   170,190,105
10. Внетренняя нижняя левая рука     228,191,74
11. Внутренняя нижняя правая рука    240,198,60
12. Левая кисть                      8,110,221
13. Правая кисть                     4,98,224
14. Левая нога                       22,173,184
15. Правая нога                      6,166,198

In [8]:
model = UNet(in_channels=3, out_channels = 16)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [9]:
import time
startTime = time.time()

train_history, val_history = train_unet(
    model,
    train_dataloader,
    val_dataloader,
    optimizer,
    device=device,
    epoch_num=25,
    save_directory="C:\\Users\\DenisovDmitrii\\Desktop\\segmentation_weits\\session2"
)

print(time.time() - startTime)


Epoch 0 of 24, train loss: -0.767
Epoch 0 of 24, val loss: -0.863


KeyboardInterrupt: 